# Full Training
Without the Trainder class

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#define tokenizer function
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

#Apply tokenizer to each row of the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Apply dynamic padding for each batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Reusing dataset glue (/home/studio-lab-user/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Prepare for training
First we need to create a dataloader that will be used to iterarte over batches. However we will need to do a few things before we can feed the tokenized dataset to the dataloader. These we taken care for us by the Trainer class

In [2]:
#Remove columns corresponding to values that the model does not expect - 'sentence1' and 'sentence2'
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

#Rename column labels to labels
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

#Set the format of the datasets so they return PyTorch tensors instead of lists
tokenized_datasets.set_format("torch")

tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

## Define Dataloaders

In [3]:
from torch.utils.data import DataLoader

In [4]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8,collate_fn=data_collator)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)

In [5]:
#spot check
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 72]),
 'token_type_ids': torch.Size([8, 72]),
 'attention_mask': torch.Size([8, 72])}

## Model

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
# Spot check by passing a batch to the model
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.0974, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


## Optimizer and Learning rate scheduler

In [8]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/home/studio-lab-user/.conda/envs/huggingface/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
#LR scheduler
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


## Training Loop

In [10]:
import torch
# Move model to GPU if available 
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [13]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/1377 [00:00<?, ?it/s]

In [14]:
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

## Evaluvation Loop

In [15]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8431372549019608, 'f1': 0.8900343642611684}